In [1]:
from bs4 import BeautifulSoup
import urllib3
import re
import json
import pandas as pd
import time
from tqdm.notebook import tqdm

In [2]:
month = 1
page = 1
topic = 'politik' # politic, wirtschaft, sport, wissen,kultur
year = 2023

In [3]:
url = f"https://www.sueddeutsche.de/archiv/{topic}/{year}/{month}/page/{page}"

In [4]:
def get_html(url):
    http = urllib3.PoolManager()
    response = http.request('GET', url,headers = {'User-agent': 'your bot 0.1'})
    soup = BeautifulSoup(response.data)
    return soup,response.status

In [5]:
soup,status = get_html(url)
all_links = []
if status==200:
    # Find all <a> tags with class="entrylist__link"
    links = soup.find_all('a', class_='entrylist__link')

    # Extract and print the href attribute from each <a> tag
    for link in links:
        href = link.get('href')
        all_links.append(href)
else:
    print('failed')

In [6]:
df = pd.DataFrame([],columns=['Url','Topic','Publish Date','Author','Title','Teaser','News'])
for link in tqdm(all_links):
    soup, status = get_html(link)
    if status==200:
        # Find all <p> tags with data-manual="paragraph"
        paragraphs = soup.find_all('p', {'data-manual': 'paragraph'})

        # Extract and print the text from each paragraph
        news = ''
        for paragraph in paragraphs:
            news = news + '\n' +paragraph.get_text(strip=True)
            
        news = news.strip()
        element = soup.find('div', {'id': 'taboola-feed-below-article', 'data-paycategory': 'free'})
        
        # Check if the element is found
        if element:
            # Extract the required attributes
            data_authors = element.get('data-authors')
            data_teaser = element.get('data-teaser')
            data_title = element.get('data-title')
            data_publishdate = element.get('data-publishdate')
            
            df.loc[len(df)] = [link,topic,data_publishdate,data_authors,data_title,data_teaser,news]
        else:
            print("Paid news",link)       
    else:
        print('Failed->',status,link)

  0%|          | 0/50 [00:00<?, ?it/s]

Paid news https://www.sueddeutsche.de/politik/meinungsfreiheit-iran-will-gesetze-verschaerfen-1.5742167
Paid news https://www.sueddeutsche.de/politik/schule-wieder-mehr-sitzenbleiber-1.5741752
Paid news https://www.sueddeutsche.de/politik/cdu-ultimatum-an-maassen-1.5741748
Paid news https://www.sueddeutsche.de/politik/verfassungsgericht-entscheidung-zur-berlin-wahl-1.5741750
Paid news https://www.sueddeutsche.de/politik/ukraine-scholz-kampfjet-1.5742078
Paid news https://www.sueddeutsche.de/politik/glosse-das-streiflicht-1.5741856
Paid news https://www.sueddeutsche.de/politik/japan-toyota-elektroautos-1.5741800
Paid news https://www.sueddeutsche.de/politik/pakistan-peschawar-moschee-explosion-anschlag-pakistanische-taliban-1.5741635
Paid news https://www.sueddeutsche.de/politik/news-nachrichten-ukraine-soli-scholz-tuerkei-1.5742176
Paid news https://www.sueddeutsche.de/politik/usa-israel-blinken-besuch-1.5742181
Paid news https://www.sueddeutsche.de/politik/faeser-hessen-spitzenkandida